# Contextualization: dynamic configuration of virtual machines
[Contextualization: dynamic configuration of virtual machines](https://journalofcloudcomputing.springeropen.com/articles/10.1186/s13677-015-0042-8)

## Abstract

New VM instances are created from static templates that contain the basic configuration of the VM to achieve elasticity with regards to capacity. Instance specific settings can be injected into the VM during the deployment phase through means of contextualization. So far this is limited to a single data source and data remains static throughout the lifecycle of the VM.\
新的VM实例是从包含VM基本配置的静态模板创建的，以实现容量弹性。特定于实例的设置可以在部署阶段通过上下文化的方式注入到VM中。到目前为止，这仅限于单个数据源，并且数据在VM的整个生命周期中都是静态的。

We present a layered approach to contextualization that supports different classes of contextualization data available from several sources. The settings are made available to the VM through virtual devices. Inside each VM data from different classes are layered on top of each other to create a unified file hierarchy.\
我们提出了一种上下文化的分层方法，它支持来自多个来源的不同类别的上下文化数据。这些设置通过虚拟设备提供给虚拟机。在每个VM中，来自不同类的数据相互层叠，以创建统一的文件层次结构。

Context data can be modified during runtime by updating the contents of the virtual devices, making our approach the first contextualization approach to natively support recontextualization. Recontextualization enables runtime reconfiguration of an executing service and can act as a trigger and key enabler of self-* techniques. This trigger provides a service with a mechanism to adapt or optimize itself in response to a changing environment. The runtime reconfiguration using recontextualization and its potential gains are illustrated in an example with a distributed file system, demonstrating the feasibility of our approach.\
上下文数据可以在运行时通过更新虚拟设备的内容来修改，这使我们的方法成为第一个本地支持重新上下文化的上下文化方法。Recontextualization支持对正在执行的服务进行运行时重新配置，可以作为self-*技术的触发器和关键推动者。这个触发器为服务提供了一种机制，以适应或优化自身以响应不断变化的环境。使用recontextualization的运行时重新配置及其潜在收益在一个分布式文件系统的例子中得到了说明，说明了我们的方法的可行性。

## Introduction

One of the key characteristics of cloud computing is rapid elasticity [1]; the ability to quickly provision or release resources assigned to a cloud service in order to respond to current demand. In the Infrastructure as a Service (IaaS) model, cloud services are normally comprised of a set of different components, each defined using a Virtual Machine (VM) template. To achieve elasticity, the capacity of a cloud service can be adapted during runtime by adjusting the number of running VM instances of each template. This makes it possible to scale each part of the service independently.\
云计算的关键特征之一是快速弹性;快速提供或释放分配给云服务的资源以响应当前需求的能力。在基础设施即服务(Infrastructure as Service, IaaS)模型中，云服务通常由一组不同的组件组成，每个组件使用虚拟机(Virtual Machine, VM)模板定义。通过调整每个模板中运行的虚拟机实例数量，可以在运行过程中调整云服务的容量，实现弹性。这使得独立扩展服务的每个部分成为可能。

Each VM instance started from a template needs to be customized with some unique settings, e.g., networking configuration to ensure each instance is assigned a unique IP-address. The settings need to be applied dynamically, normally done as part of the VM boot process. This boot-time customization process is called **contextualization** [2, 3].\
从模板启动的每个虚拟机实例都需要定制一些独特的设置，例如，网络配置，以确保每个实例都被分配一个唯一的ip地址。需要动态应用这些设置，通常是在VM引导过程中进行的。这种引导时定制过程称为**上下文化**[2,3]。

Several different usage scenarios that join isolated cloud infrastructures together into a larger unified infrastructure are being considered [4, 5]. Conceptually, a virtual market of cloud resources not limited by technological boundaries would offer diversity in terms of (but not limited to) pricing, availability, and a choice of different geographical locations to use for hosting. Cloud infrastructures commonly offer supporting services such as network based storage, and today’s major infrastructure providers (IPs) such as Amazon wrap the infrastructure specific functionality into pre-configured VM templates. In a multi-cloud scenario the VM templates need to be generic enough to be deployed across a wide range of infrastructures. **Therefore, the contextualization stage is needed to support interactions with any infrastructure-specific services or settings.**\
正在考虑将隔离的云基础设施连接到一个更大的统一基础设施的几种不同的使用场景[4,5]。从概念上讲，不受技术边界限制的云资源虚拟市场将在(但不限于)价格、可用性和用于托管的不同地理位置的选择方面提供多样性。云基础设施通常提供支持服务，如基于网络的存储，今天的主要基础设施提供商(ip)，如Amazon，将基础设施特定的功能包装到预先配置的VM模板中。在多云场景中，VM模板需要足够通用，以便跨各种基础设施部署。因此，需要上下文化阶段来支持与任何特定于基础设施的服务或设置的交互。

VMs can be migrated between physical hosts without being restarted using live-migration. This enables an IP to re-distribute the current VM load across available server resources during runtime. Because the VM is not restarted during the migration, the contextualization procedure is not triggered post-migration. Therefore, any VM customization (such as loading infrastructure specific settings) will remain unchanged.\
虚拟机热迁移无需重启即可在物理主机间迁移。这使得IP能够在运行时跨可用服务器资源重新分配当前VM负载。由于迁移过程中未重启虚拟机，所以迁移后不会触发上下文化过程。因此，任何虚拟机定制(如加载基础架构特定设置)将保持不变。

In our earlier work [6] we present and define recontextualization, an emerging technology to allow contextual data inside running VMs to be updated during runtime. This work extends on earlier work on recontextualization and contextualization [3] and presents a novel layered based approach to contextualization. In this approach, multiple independent sets of contextual data are merged into a unified structure realized by a custom file system whose data sets can be dynamically added or removed during runtime. This paper presents the requirements, design, and evaluation of a system that support this multi-layered approach to contextualization, which is the first contextualization solution to natively support recontextualization. It is worth noting that the research presented in this paper uses a scientific methodology based on functional prototyping and system performance analysis. We do not use mathematical modelling. Additionally and for clarity’s sake, the results of this research aims to facilitate and enhance current cloud provider interoperability beyond the state of the art but we do not claim to solve all problems of VM or IaaS interoperability.\
在我们早期的工作[6]中，我们提出并定义了recontextualization，这是一种新兴的技术，允许在运行时更新运行中的vm中的上下文数据。这项工作扩展了早期关于重新上下文化和上下文化[3]的工作，并提出了一种新的基于分层的上下文化方法。在这种方法中，多个独立的上下文数据集被合并成一个统一的结构，由一个自定义文件系统实现，该文件系统的数据集可以在运行时动态添加或删除。本文提出了一个支持这种多层上下文化方法的系统的需求、设计和评估，这是第一个本地支持重新上下文化的上下文化解决方案。值得注意的是，本文的研究采用了基于功能原型和系统性能分析的科学方法。我们不使用数学模型。此外，为了明确起见，本研究的结果旨在促进和增强现有云提供商的互操作性，超出现有技术水平，但我们并不是说可以解决VM或IaaS互操作性的所有问题。

The rest of this article is organized as follows; In Section ‘Background and related work’ the background of topics related to the field of contextualization and recontextualization are presented. Section ‘Contextualization’ presents an overview of contextualization including challenges and a summary of our earlier work on the subject. The corresponding information for recontextualization is presented in Section ‘Recontextualization’. Section ‘Context-aware lifecycle management’ shows how the two techniques are used in conjunction to provide adaptable usage throughout the lifecycle of a service. The unified approach is demonstrated and evaluated in Section ‘Functional evaluation’. Finally, conclusions and future work are presented in Section ‘Conclusions and future work’.\
本文的其余部分组织如下;在“背景和相关工作”一节中，介绍了语境化和再语境化领域相关课题的背景。“语境化”部分介绍了语境化的概述，包括挑战和对我们在这个主题上早期工作的总结。recontextualization的相应信息在Section ' recontextualization '中显示。“上下文感知生命周期管理”小节展示了如何结合使用这两种技术来在服务的整个生命周期中提供适应性的使用。在“功能评估”一节中演示和评估了统一的方法。最后，结论和未来的工作在“结论和未来的工作”部分提出。

## Contextualization

### Challenges
There are a number of challenges that make the contextualization of cloud services a non-trivial affair. In our previous work we argued that contextualization in cloud computing is a highly pervasive key technological requirement of any cloud service, where elastic resource management is critical to the on-demand scalability of a service [3]. The holistic nature of the services deployed on clouds makes it difficult to provide flexible generic and open tools without limiting the heterogeneity of supported services. We identified three inherent challenges to providing elasticity through contextualization where VMs are added and removed during service operation.\
有许多挑战使得云服务的环境化成为一件重要的事情。在我们之前的工作中，我们认为云计算中的上下文化是任何云服务的一个高度普及的关键技术需求，其中弹性资源管理对服务[3]的按需可伸缩性至关重要。部署在云上的服务的整体性使得在不限制所支持服务的异构性的情况下提供灵活的通用和开放的工具变得困难。我们确定了通过上下文化(在服务操作期间添加和删除vm)提供弹性的三个内在挑战。

The first challenge to overcome is the complete contextualization of cloud services across all classifications within the cloud ecosystem: Software as a Service (SaaS), Platform as a Service (PaaS), and Infrastructure as a Service (IaaS) [1, 28]. These classifications refer to:\
要克服的第一个挑战是云生态系统中所有类别云服务的完整上下文化:软件即服务(SaaS)、平台即服务(PaaS)和基础设施即服务(IaaS)[1,28]。这些分类是指:

* SaaS: web-based applications and services hosted on-line, usually reachable from standard interfaces such as a web browser.\
SaaS:在线托管的基于web的应用程序和服务，通常可以通过web浏览器等标准界面访问。

* PaaS: systems that offer deployment of applications designed for execution on a specific platform or software environment. For example, the opportunity to upload a Java archive and have it executed on remote resources.\
PaaS:提供在特定平台或软件环境上执行的应用程序部署的系统。例如，可以上传Java归档并在远程资源上执行它。

* IaaS: infrastructures that offer the provisioning of remote resources on which the consumer can execute arbitrary software as if the resources were dedicated hardware servers. Virtualization is usually used as a layer between the hardware and the consumer software systems, which makes it possible for the consumer to design the software stack from the operating system upwards.\
IaaS:提供远程资源供应的基础设施，消费者可以在其上执行任意软件，就像这些资源是专用的硬件服务器一样。虚拟化通常用作硬件和消费者软件系统之间的一层，这使得消费者可以从操作系统向上设计软件堆栈。

In this work, we focus on IaaS contextualization of VMs comprising a cloud service. We touch upon using the suggested approach to access platform software services such as network based storage or database services that could be considered stand-alone PaaS services. In IaaS, the challenge pertains to low-level contextualization of virtual resources, as found in IPs, where virtual devices require context to enable VMs to be bootstrapped to existing virtual infrastructures. This approach has been partially explored by RESERVOIR [5].\
在这项工作中，我们关注由云服务组成的虚拟机的IaaS上下文化。我们谈到了使用建议的方法来访问平台软件服务，例如可以视为独立PaaS服务的基于网络的存储或数据库服务。在IaaS中，挑战涉及虚拟资源的底层上下文化，如在ip中，虚拟设备需要上下文以使虚拟机引导到现有的虚拟基础设施。RESERVOIR[5]对这种方法进行了部分探索。

The second challenge to overcome is contextualization across multiple IaaS domains for reasons of interoperability. Many IaaS providers offer platform services that are not interoperable with those of other providers. Also, as contextualization of VMs is performed as part of service development, the service will be customized to a single provider only. To solve this challenge, we need to extend contextualization to support run-time recontextualization.\
第二个需要克服的挑战是基于互操作性的跨多个IaaS域的上下文化。许多IaaS提供者提供的平台服务不能与其他提供者的平台服务互操作。另外，由于vm的上下文化是作为服务开发的一部分来执行的，因此服务将只针对单个提供者进行定制。为了解决这个挑战，我们需要扩展上下文化以支持运行时重新上下文化。

The third challenge pertains to a set of functional requirements for real world clouds and their impact on contextualization. Notable among these are end-to-end security through contextualization mechanisms that support a Virtual Private Network (VPN) overlay and software license management systems. Both of these have unique contextualization requirements: contextualization must be secure with no VPN keys stored unless in use; and contextualization that is able to accommodate license protected software and licensing tokens.\
第三个挑战涉及现实世界云的一组功能需求及其对上下文化的影响。其中值得注意的是，通过支持虚拟专用网(VPN)覆盖层和软件许可证管理系统的上下文化机制实现的端到端安全性。这两种方法都有独特的上下文化要求:上下文化必须是安全的，除非使用，否则不存储VPN密钥;以及能够容纳许可保护软件和许可令牌的上下文化。

In our previous work [6] we discuss the nature of contextualization in the light of the OPTIMIS project [4]. We present details of the architecture and implementation of our tool for the contextualization of platform level services as well as virtual infrastructure. We discuss the implication of contextualization in clouds, the motivation behind our work and suggest a landscape for the evolution of contextualization tools across all classes of clouds within the ecosystem of the future. We contribute to both the image and instance level contextualization of VM’s and illustrate the potential effectiveness of our tool through a simple use case.\
在我们之前的工作[6]中，我们根据OPTIMIS项目[4]讨论了上下文化的本质。我们将详细介绍平台级服务和虚拟基础设施上下文化工具的体系结构和实现。我们讨论了云环境化的含义，我们工作背后的动机，并提出了未来生态系统中所有类型云环境化工具的发展前景。我们贡献了VM的映像和实例级上下文化，并通过一个简单的用例说明了我们的工具的潜在有效性。

In addition, as part of our previous work on contextualization, a proof of concept contextualization tool was developed and its performance tested. This prototype was used to confirm the validity of our contextualisation approach on a cloud testbed. This provided evidence on the potential performance of our approach for contextualization with regards to preparing generic VM images of sizes in the range of 1-5 GB in increments of 1 GB and with varying numbers of concurrent user requests from 10-100, for the purpose of creating ISO CD images containing 1 MB of context data. The results showed adequate scalability and response time over ten iterations of the experiment with minimal variance.\
此外，作为我们之前关于上下文化工作的一部分，我们开发了一个概念上下文化证明工具，并测试了它的性能。该原型被用于在云测试平台上验证我们的情境化方法的有效性。这对潜在的性能提供了证据的方法把握语境对于准备通用虚拟机镜像的大小在1 - 5 GB的增量1 GB的范围和不同数量的并发用户请求从10 - 100,为了创建ISO CD映像包含1 MB的上下文数据。结果表明，在最小方差的十次迭代实验中，有足够的可伸缩性和响应时间。

# Towards a Contextualization Solution for Cloud Platform Services
[Towards a Contextualization Solution for Cloud Platform Services](https://ieeexplore.ieee.org/document/6133160)

## Abstract

We propose a cloud contextualization mechanism which operates in two stages, contextualization of VM images prior to service deployment (PaaS level) and selfcontextualization of VM instances created from the image (IaaS level). The contextualization tools are implemented as part of the OPTIMIS Toolkit, a set of software components for simplified management of cloud services and infrastructures. We present the architecture of our contextualization tools and the feasibility of our contextualization mechanism is demonstrated in a threetier web application scenario. Preliminary performance results suggest acceptable performance and scalability of our prototype.\
我们提出了一种云上下文化机制，它分两个阶段运行，即服务部署之前的VM映像的上下文化(PaaS级别)和从映像创建的VM实例的自上下文化(IaaS级别)。上下文化工具作为OPTIMIS Toolkit的一部分实现，OPTIMIS Toolkit是一组简化云服务和基础设施管理的软件组件。我们提出了我们的上下文化工具的架构，并在一个三层web应用场景中演示了我们的上下文化机制的可行性。初步性能结果表明，我们的原型的性能和可伸缩性可以接受。

## I. INTRODUCTION

Modern virtualization technologies enable rapid provisioning of Virtual Machines (VMs) and thus allow cloud services to scale up and down on-demand. This elasticity [6], [8] comes with a new set of challenges for dynamic service configuration. **Contextualization is a set of processes and mechanisms that enable a service to scale elastically alongside the resources and software that support it through the orchestration of these dependencies toward the common goals of the service.** We focus on horizontal elasticity where scaling is achieved by adding or removing VMs to a service during its operation. The related case of vertical elasticity, i.e. application scaling through VM resizing, is much easier from a contextualization perspective. For horizontal elasticity scenarios, predefined yet flexible contextualization mechanisms enable the VMs of a service to self-discover and communicate.\
现代虚拟化技术支持快速提供虚拟机(vm)，从而允许云服务按需伸缩。这种弹性[6]和[8]为动态服务配置带来了一组新的挑战。上下文化是一组流程和机制，通过编排这些依赖关系，使服务能够与支持它的资源和软件一起灵活地伸缩，以达到服务的共同目标。我们关注水平弹性，即通过在服务运行期间向其添加或删除vm来实现伸缩性。垂直弹性的相关情况，即通过VM调整大小的应用程序伸缩，从上下文的角度来看要容易得多。对于水平弹性场景，预定义但灵活的上下文化机制使服务的vm能够自我发现和通信。

We identify three main challenges in enabling elasticity through contextualization where VMs are added and removed during service operation: i) contextualization support offered by the Platform as a Service (PaaS) layer, to replace the traditional approach that requires complex and time consuming manipulation of VM images as part of development of each Software as a Service (SaaS) solution; ii) contextualization of services that are deployed across multiple Infrastructure as a Service (laaS) providers; and iii) contextualization with support for functional requirements such as secure network overlays and the incorporation of licence-protected software in services.\
我们确定了通过上下文化(在服务运行期间添加和删除vm)实现弹性的三个主要挑战:i)平台即服务(PaaS)层提供的环境化支持，以取代传统方法，传统方法需要在每个软件即服务(SaaS)解决方案的开发过程中对虚拟机映像进行复杂且耗时的操作;ii)跨多个基础设施作为服务(laaS)提供者部署的服务的环境化;以及iii)支持功能需求的环境化，如安全网络覆盖和在服务中整合受许可保护的软件。

Our contextualization tools are implemented as part of the OPTIMIS Toolkit, a set of software components for simplified management of cloud services and infrastructures. The feasibility of our contextualization mechanisms is demonstrated in a three-tier web application scenario, where the database and application server tiers both are replicated and scaled elastically. The contextualization mechanisms enable elasticity for this application such that instances of database and application server can be transparently added and removed from the service during operation. We also provide results that demonstrate the scalability and performance of our mechanism in a multi-user cloud environment.\
我们的环境化工具是作为OPTIMIS Toolkit的一部分实现的，OPTIMIS Toolkit是一组简化云服务和基础设施管理的软件组件。我们的上下文化机制的可行性在一个三层的web应用程序场景中得到了演示，其中数据库和应用服务器层都被弹性地复制和伸缩。上下文化机制为这个应用程序提供了弹性，这样数据库和应用服务器的实例就可以在操作期间透明地从服务中添加和删除。我们还提供了在多用户云环境中演示我们的机制的可伸缩性和性能的结果。

The remainder of this paper is structured as follows; Section 2 discusses the challenges within contextualization and motivates our work. Section 3 describes our proposed contextualization architecture for OPTIMIS, an illustrative use case and preliminary performance results. Section 4 discusses related work and finally Section 5 presents our conclusions and future work.\
本文的其余部分结构如下:第2节讨论了情境化中的挑战和我们工作的动力。第3节描述了我们为OPTIMIS提出的上下文化架构，一个说明性的用例和初步的性能结果。第4节讨论了相关工作，最后第5节提出了我们的结论和未来的工作。

## SECTION III.The Contextualization Tools
### A. Contextualization Architecture
The contextualization tools are comprised of a core component, the VM Contextualizer and a repository of scripts. This component provides an interface for consuming service context data, such as: security certificates, VPN hostnames, VPN DNS and Gateway IP addresses, mount points for network data stores, monitoring manager hostnames, offline software license tokens and a list of software dependencies. The VM Contextualizer provides two capabilities. The first is a bootstrap mechanism to prepare a VM image for receipt of context agnostic of the operating system type used. The second capability provides a mechanism for creating ISO CD ROM images that contain context data and a data processing script for the manipulation of the data into a format suitable for consumption at runtime when the ISO image is mounted. The VM Contextualizer mounts a VM image and modifies it to include an assortment of bespoke scripts that interact with the guest operating system, service and service dependencies at boot time, preparing the image to receive context in a reusable fashion. At run time these scripts access contextualization data held within this ISO image, as per the OVF recommendation [11], giving a VM its identity.\
上下文化工具由核心组件、VM Contextualizer和脚本存储库组成。该组件提供了一个使用服务上下文数据的接口，例如:安全证书、VPN主机名、VPN DNS和网关IP地址、网络数据存储的挂载点、监控管理器主机名、离线软件许可令牌和软件依赖列表。VM Contextualizer提供两种功能。第一个是引导机制，用于准备一个VM映像，以接收与所使用的操作系统类型无关的上下文。第二个功能提供了一种机制，用于创建包含上下文数据的ISO CD ROM映像，以及一个数据处理脚本，用于将数据操作成一种适合于挂载ISO映像时在运行时使用的格式。VM Contextualizer挂载一个VM映像并对其进行修改，使其包含一组定制的脚本，这些脚本在引导时与客户操作系统、服务和服务依赖项交互，使映像以可重用的方式接收上下文。在运行时，这些脚本根据OVF建议[11]访问保存在这个ISO映像中的上下文化数据，从而为VM提供它的标识。

The inclusion of the ISO image as a mechanism to store contextualization information provides a facility to separate the contextualization data from the VM image. This removes the time consuming need to create multiple unique VM images for each VM that is required to be contextualized, while also improving the security of the contextualization process as security certificates are not stored in the VM image itself but instead stored in the ISO image when it is dynamically generated. The inclusion of a script to process the contextualization data provides an approach to store the data agnostic of the operating system, service and service dependencies used.\
将ISO映像作为一种存储上下文化信息的机制包含进来，提供了一种将上下文化数据与VM映像分离的工具。这消除了费时需要创建多个独特的每个VM虚拟机镜像,需要语境化,同时改善语境化过程的安全安全证书不是存储在VM图像本身,而是存储在ISO映像时动态生成。包含处理上下文化数据的脚本提供了一种方法来存储不依赖于所使用的操作系统、服务和服务依赖项的数据。

The VM Contextualizer uses parts of QEMU [12], a generic and open source machine emulator and virtualizer to manipulate images. QEMU provides a tool named “qemu-img” that enables the conversion of virtual machine images. In addition, Linux system tools (such as “mount”, “iosetup” and “kpartx”) are used to mount VM images as loop devices for write manipulation. For the creation of ISO images, the Linux system tool “mkisofs” is used in addition to the previously mentioned tools to create and modify ISO images.\
VM Contextualizer使用QEMU[12]的部分内容来操作映像，QEMU[12]是一个通用的开源计算机仿真器和虚拟器。QEMU提供了一个名为“QEMU -img”的工具，它支持虚拟机映像的转换。此外，Linux系统工具(如“mount”、“iosetup”和“kpartx”)用于挂载虚拟机镜像，作为循环设备进行写操作。对于创建ISO镜像，除了前面提到的工具外，还使用Linux系统工具“mkisofs”来创建和修改ISO镜像。

Our contextualization research is performed in the context of the OPTIMIS Toolkit [4], a set of software components aimed to simplify and optimize the construction, deployment, and operation of services (at the SaaS and PaaS level) as well as the operation of virtualized hardware needed to deliver these services (IaaS level). Although OPTIMIS targets scenarios and capabilities somewhat different to the ones provided by current PaaS and IaaS providers, the OPTIMIS contextualization requirements are general and the solution should be applicable in a wide range of cloud providers.\
我们的情境化研究的上下文中执行的优化工具箱[4],一组软件组件旨在简化和优化建设、部署和运行服务(SaaS和PaaS层)以及虚拟硬件的操作需要提供这些服务(IaaS)。虽然OPTIMIS的目标场景和功能与当前PaaS和IaaS提供商提供的场景和功能有些不同，但OPTIMIS的上下文化需求是通用的，该解决方案应该适用于各种云提供商。

### B. Service and Toolkit Component Contextualization
B.服务和工具箱组件上下文化

Figure 1 illustrates the instance-level contextualization process of a VM at the beginning of its execution. During the boot sequence of a VM the contextualization tools mount an ISO CD image that contains contextualization data and a script to process this data into a useable form. This script communicates through a known interface to the OS specific contextualisation scripts embedded in the VM image. These OS specific scripts manipulate configuration files of associated OPTIMIS components and software dependencies that support the end user's service(s), setting their context. The scripts can remain in a daemon-like mode for a component or software dependency that requires continuous updates to its context, enabling dynamic reconfiguration and instant discovery of elastic resources, if needed.\
图1演示了一个虚拟机开始执行时的实例级上下文化过程。在虚拟机的引导序列期间，上下文化工具会挂载一个包含上下文化数据的ISO CD镜像和一个将该数据处理为可用形式的脚本。该脚本通过一个已知的接口与嵌入在VM映像中的特定于操作系统的上下文脚本进行通信。这些特定于操作系统的脚本操作相关的OPTIMIS组件的配置文件和支持最终用户服务的软件依赖项，设置它们的上下文。对于需要不断更新其上下文的组件或软件依赖项，脚本可以保持类似守护进程的模式，从而在需要时支持动态重新配置和即时发现弹性资源。

There are several problems to solve regarding contextualization of OPTIMIS system level components, such as those associated with license management and cloud security. The following subsections outline these issues:\
关于OPTIMIS系统级组件的上下文化有几个问题需要解决，比如那些与许可证管理和云安全相关的问题。以下各节概述了这些问题:

#### 1) License Management
Access to licenses for authorizing the execution of an application in a Cloud beyond the administrative domain of the site running the license server usually leads to applications aborting during startup because of unreachable license servers, e.g. due to firewall issues. In OPTIMIS we use a prototype for software licensing developed in the European project SmartLM [2], which provides licensing technology for location independent application execution. Separation of authorization for license usage and authorization for application execution on the one hand and software tokens that carry the authorization information on the other hand provide the necessary flexibility for licenses following applications into Clouds. It is one of the tasks of the VM Contextualizer to retrieve and embed a license token into the VM hosting the application. In case of multiple applications in a VM, required e.g. for a workflow, the VM Contextualizer assures that all required licenses are in place when the applications start up. No communication between the application and the license server that issued the token is required at runtime. Additional approaches will be implemented enhancing the SmartLM solution: (i) dynamic deployment of a trusted instance managing a number of tokens for one or multiple applications and (ii) dynamic deployment of a full license service with a subset of the licenses available at the home organisation of the user. The configuration of the dynamically deployed license service will be managed by the VM Contextualizer. This approach is especially useful when the same Cloud resources are used over a longer period of time for running license protected applications. In the first approach the contextualizer tools are responsible for configuring and deploying the trusted instance for the respective network environment and to transfer tokens.\
在运行许可证服务器的站点的管理域之外访问用于授权执行云中的应用程序的许可证，通常会导致应用程序在启动期间由于无法访问许可证服务器而中止，例如由于防火墙问题。在OPTIMIS中，我们使用了欧洲项目SmartLM[2]开发的软件许可原型，该项目为位置独立的应用程序执行提供了许可技术。许可证使用授权和应用程序执行授权的分离，以及另一方面携带授权信息的软件令牌，为进入云的应用程序之后的许可证提供了必要的灵活性。VM Contextualizer的任务之一是检索一个许可令牌并将其嵌入到承载应用程序的VM中。在一个VM中有多个应用程序的情况下，例如一个工作流所需的情况下，VM Contextualizer确保在应用程序启动时所有所需的许可证都到位。在运行时，应用程序和发出令牌的许可服务器之间不需要通信。还将实施其他方法来增强SmartLM解决方案:(i)动态部署一个可信任的实例，管理一个或多个应用程序的多个令牌;(ii)动态部署一个完整的许可证服务，其中包含用户所在机构可用的许可证的子集。动态部署的许可服务的配置将由VM Contextualizer管理。当长时间使用相同的云资源运行受许可证保护的应用程序时，这种方法特别有用。在第一种方法中，上下文生成器工具负责为各自的网络环境配置和部署可信实例，并传输令牌。

#### 2) Cloud Security
Each instance of a VM requires specific security customisations based on the service it provides and its threat profile. For example, the firewall rules specific to a web server VM is different from that of a database VM and these variations are handled by the contextualization tools. In addition, the OPTIMIS Data Manager provides a means of provisioning secure encrypted storage devices for VMs, where the decryption keys are stored outside an IP. The specifics of these secure device configurations are different across various VMs and are set by the contextualization steps. Various Identity and Access Management (IdAM) components that need to be installed, along with policies specified at the VM endpoints, are also set by this component. If required by the end user or SP, other security mechanisms like Intrusion Prevention Software can also be instantiated and customised by similar mechanisms.\
每个虚拟机实例都需要根据其提供的服务和威胁配置文件进行特定的安全定制。例如，特定于web服务器VM的防火墙规则与数据库VM的不同，这些变化由上下文化工具处理。此外，OPTIMIS数据管理器提供了一种为虚拟机提供安全加密存储设备的方法，其中解密密钥存储在IP之外。这些安全设备配置的细节在不同的vm中是不同的，并由上下文化步骤进行设置。该组件还设置了需要安装的各种身份和访问管理(IdAM)组件，以及在VM端点指定的策略。如果最终用户或SP需要，其他安全机制(如入侵防御软件)也可以通过类似的机制实例化和定制。

### C. A Contextualization Use Case

At the IP level, predefined context from a SP is applied to a VM as it is brought online. This does not require communication with any IP level component and addresses problematic re-contextualization of VMs at runtime. A three tier web application demonstrates the OPTIMIS toolkit in operation and the complexities involved in the contextualization of VMs. Figure 2 illustrates the relationship between the contextualization data and a VM. From the figure it can be seen that each layer of the service forms a cluster of cooperating resources that rely on a head node to provide information on the state of VMs and balance load. Each new VM brought online contains information about the head node to which it is to register for active duty in addition to other VMs to which it must communicate to perform its role. The information is stored within the contextualization data of the VM, which includes a subset of the contextualization data from the head node and can reference other sources of information. These other sources of information can update the contextualization data continuously during runtime if needed. This enables VMs to be taken offline without disturbing the operation of a service.\
在IP级别，当VM上线时，预定义的来自SP的上下文被应用到VM。这不需要与任何IP级组件通信，并在运行时解决vm的问题重新上下文化。一个三层的web应用程序演示了运行中的OPTIMIS工具包以及虚拟机上下文化所涉及的复杂性。图2说明了上下文化数据和VM之间的关系。从图中可以看出，服务的每一层构成了一个协作资源集群，依赖于一个头节点来提供关于vm状态的信息和平衡负载。每个上线的新VM包含关于它要注册到的主节点的信息，以及它必须通信到的其他VM来执行其角色的信息。信息存储在VM的上下文化数据中，其中包括来自头节点的上下文化数据的子集，并可以引用其他信息源。如果需要，这些其他信息源可以在运行时不断更新上下文化数据。这使得虚拟机可以离线而不影响服务的运行。